In [11]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from jupyter_dash import JupyterDash
from datetime import datetime

dash_colors = {
    'background': 'white',
    'text': '#BEBEBE',
    'grid': '#333333',
    'red': '#BF0000',
    'blue': '#466fc2',
    'green': '#5bc246',
#     'indicator_background': '#2d6a8a'
    'indicator_background': '#e4d2cf'
}

# Get the data into the app
# confirmed = pd.read_csv("data/confirmed_global.csv")
# deaths = pd.read_csv('data/deaths_global.csv')
# recovered = pd.read_csv("data/recovered_global.csv")

confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

# Unpivot the data frames
total_confirmed = confirmed.melt(
  id_vars = ["Province/State", "Country/Region", "Lat", "Long"],
  value_vars = confirmed.columns[4:],
  var_name = "date",
  value_name = "confirmed"
)

total_deaths = deaths.melt(
  id_vars = ["Province/State", "Country/Region", "Lat", "Long"],
  value_vars = deaths.columns[4:],
  var_name = "date",
  value_name = "deaths"
)

total_recovered = recovered.melt(
  id_vars = ["Province/State", "Country/Region", "Lat", "Long"],
  value_vars = recovered.columns[4:],
  var_name = "date",
  value_name = "recovered"
)

# Merge data frames
covid_data = total_confirmed.merge(
  right = total_deaths,
  how = "left",
  on = ["Province/State", "Country/Region", "date", "Lat", "Long"]
).merge(
  right = total_recovered,
  how = "left",
  on = ["Province/State", "Country/Region", "date", "Lat", "Long"]
)

# Wrangle data
covid_data["recovered"] = covid_data["recovered"].fillna(0)
covid_data["active"] = covid_data["confirmed"] - covid_data["deaths"] - covid_data["recovered"]
covid_data["date"] = pd.to_datetime(covid_data["date"])

# Daily totals
covid_data_1 = covid_data.groupby(["date"])[["confirmed", "deaths", "recovered", "active"]].sum().reset_index()

# Create dict of list
covid_data_list = covid_data[["Country/Region", "Lat", "Long"]]
dict_of_locations = covid_data_list.set_index("Country/Region")[["Lat", "Long"]].T.to_dict("dict")

# Instanciate the app
app = JupyterDash( meta_tags = [{"name": "viewport", "content": "width=device-width"}])

# Build the layout
app.layout = html.Div(
	children = [

		# (First row) Cards: COVID-19 CASES - Global cases - Global deaths - Global active
		html.Div(
			children = [
                html.Div(
					children = [
                        html.Br(),
						# (Column 1): COVID-19 CASES
						html.H2(
							children = "COVID-19 CASES",
							style = {
								"textAlign": "center",
								"color": "white",
                                
							}
						),
					],style={'backgroundColor':'#34495c'},
					className = "card_container three columns"
				),
				# (Column 2): Global cases
				html.Div(
					children = [
						# Title
						html.H6(
							children = "Worldwide cases",
							style = {
								"textAlign": "center",
								"color": "white"
							}
						),
						# Total value                         
						dcc.Graph(
							id = "confirmed_ind",
							config = {
								"displayModeBar": False
							},
							style = {
								"textAlign": "center",
								"color": "orange",
								"fontSize": 30
							}
						),
						# New cases
					],
					className = "card_container three columns"
				),
				# (Column 3): Global deaths
				html.Div(
					children = [
						# Title
						html.H6(
							children = "Worldwide deaths",
							style = {
								"textAlign": "center",
								"color": "white"
							}
						),
						# Total value
						dcc.Graph(
							id = "death_ind",
							config = {
								"displayModeBar": False
							},
							style = {
								"textAlign": "center",
								"color": "red",
								"fontSize": 30
							}
						),
					],
					className = "card_container three columns"
				),

				# (Column 4): Global active
				html.Div(
					children = [
						# Title
						html.H6(
							children = "Worldwide active",
							style = {
								"textAlign": "center",
								"color": "white"
							}
						),
						# Total value
						dcc.Graph(
							id = "active_ind",
							config = {
								"displayModeBar": False
							},
							style = {
								"textAlign": "center",
								"color": "#e55467",
								"fontSize": 30
							}
						),

					],
					className = "card_container three columns"
				)
			],
			className = "row flex-display"
		),
        
                # (Second Row) slider
		html.Div(
			children = [
        html.Div(html.Div(dcc.Slider(id='date_slider',
            min=list(range(len(covid_data_1['date'].unique())))[0],
            max=list(range(len(covid_data_1['date'].unique())))[-1],
            value=list(range(len(covid_data_1['date'].unique())))[-1],
            marks={(idx): {'label': date.format(u"\u2011", u"\u2011") if
                date[4:6] in ['01', '15'] else '', 'style':{'transform': 'rotate(30deg) translate(0px, 7px)'}} for idx, date in
                enumerate(sorted([item.strftime("%m{}%d{}%Y") for
                item in pd.Series(covid_data_1['date'].unique())],
                key=lambda date: datetime.strptime(date, '%m{}%d{}%Y')))},  # for bi-monthly marks
            step=1,
            vertical=False,
            updatemode='mouseup'),
        style={'width': '100%', 'float': 'left'}),  # width = 1 - (100 - x) / x
        style={'width': '95%', 'float': 'right'}),  # width = x
			],
			className = "row flex-display"
		),  
        
        
		# (Third row): pie chart - DropDown box - Line chart
		html.Div(
			children = [
				# (Column 1) 
                html.Div(
					children = [
						# pie chart
						dcc.Graph(
							id = "pie_chart",
							config = {
								"displayModeBar": "hover"
							}
						)
					],
					className = "create_container four columns",
					style = {'width':'40%',
						"maxWidth": "400px"
					}
				),
				# (Column 2) DropDown box
				html.Div(
					children = [
						# (Row 1) Country selector
						html.P(
							children = "Select Country: ",
							className = "fix_label",
							style = {
								"color": "white"
							}
						),
						dcc.Dropdown(
							id = "w_countries",
							multi = True,
							searchable = True,
							value = ["Egypt"],
							placeholder = "Select Country",
							options = [{"label": c, "value": c} for c in (covid_data["Country/Region"].unique())],
							className = "dcc_compon"
						),
						# (Row 2) New cases title
						html.P(
							children = "New cases: " + " " + str(covid_data["date"].iloc[-1].strftime("%B %d, %Y")),
							className = "fix_label",
							style = {
								"textAlign": "center",
								"color": "white"
							}
						),
						# (Row 3) Radio Items
                        dcc.RadioItems(
                        id='column_select',
                        options=[{'label': i, 'value': i} for i in ['confirmed', 'active', 'recovered', 'deaths']],
                        value='confirmed', 
                        labelStyle={'float': 'center', 'display': 'inline-block'},
                        style={'textAlign': 'center',
                            'color': dash_colors['text'],
                            'width': '100%',
                            'float': 'center',
                            'display': 'inline-block'
                            }),
                        

					],style={'width':'30%',"textAlign": "center"},
					className = "create_container three columns"
				),

				# (Columns 3) Line plot
				html.Div(
					children = [
						dcc.Graph(
							id = "line_chart",
							config = {
								"displayModeBar": "hover"
							}
						)
					],
					className = "create_container five columns"
				)
			],
			className = "row flex-display"
		),
        

        # (Fourth Row) Map
		html.Div(
			children = [
				html.Div(
					children = [
						dcc.Graph(
							id = "map_chart",
							config = {
								"displayModeBar": "hover"
							}
						)
					],
					className = "create_container1 twelve columns"
				)
			],
			className = "row flex-display"
		)
	],
	id = "mainContainer",
	style = {
		"display": "flex",
		"flex-direction": "column"
	}
)

# Build the callbacks
@app.callback(
    Output('confirmed_ind', 'figure'),
    Input('date_slider', 'value')
)
def confirmed(date_index):
    date = covid_data['date'].unique()[date_index]
    covid_data_filtered = covid_data[covid_data['date'] <= date]
    value = covid_data_filtered['confirmed'].sum()
#     delta = covid_data_filtered['confirmed'].sum()
    return {
            'data': [{'type': 'indicator',
                    'mode': 'number',
                    'value': value,
#                     'delta': {'reference': delta,
#                               'valueformat': ',g',
#                               'relative': False,
#                               'increasing': {'color': dash_colors['blue']},
#                               'decreasing': {'color': dash_colors['green']},
#                               'font': {'size': 25}},
                    'number': {'valueformat': ',',
                              'font': {'size': 30}},
                    'domain': {'y': [0, 1], 'x': [0, 1]}}],
            'layout': go.Layout(
#                 title={'text': "CUMULATIVE CONFIRMED"},
                font=dict(color="orange"),
                paper_bgcolor = "#1f2c56",
                plot_bgcolor = "#1f2c56",
                height=40,
#                 shape
                
                )
            }

@app.callback(
    Output('death_ind', 'figure'),
    Input('date_slider', 'value')
)
def death(date_index):
    date = covid_data['date'].unique()[date_index]
    covid_data_filtered = covid_data[covid_data['date'] <= date]
    value = covid_data_filtered['deaths'].sum()
#     delta = covid_data_filtered['deaths'].sum()
    return {
            'data': [{'type': 'indicator',
                    'mode': 'number',
                    'value': value,
#                     'delta': {'reference': delta,
#                               'valueformat': ',g',
#                               'relative': False,
#                               'increasing': {'color': dash_colors['blue']},
#                               'decreasing': {'color': dash_colors['green']},
#                               'font': {'size': 25}},
                    'number': {'valueformat': ',',
                              'font': {'size': 30}},
                    'domain': {'y': [0, 1], 'x': [0, 1]}}],
            'layout': go.Layout(
#                 title={'text': "CUMULATIVE CONFIRMED"},
                font=dict(color="red"),
                paper_bgcolor = "#1f2c56",
                plot_bgcolor = "#1f2c56",
                height=40,
#                 shape
                
                )
            }

@app.callback(
    Output('active_ind', 'figure'),
    Input('date_slider', 'value')
)
def active(date_index):
    date = covid_data['date'].unique()[date_index]
    covid_data_filtered = covid_data[covid_data['date'] <= date]
    value = covid_data_filtered['active'].sum()
#     delta = covid_data_filtered['active'].sum()
    return {
            'data': [{'type': 'indicator',
                    'mode': 'number',
                    'value': value,
#                     'delta': {'reference': delta,
#                               'valueformat': ',g',
#                               'relative': False,
#                               'increasing': {'color': dash_colors['blue']},
#                               'decreasing': {'color': dash_colors['green']},
#                               'font': {'size': 25}},
                    'number': {'valueformat': ',',
                              'font': {'size': 30}},
                    'domain': {'y': [0, 1], 'x': [0, 1]}}],
            'layout': go.Layout(
#                 title={'text': "CUMULATIVE CONFIRMED"},
                font=dict(color="#e55467"),
                paper_bgcolor = "#1f2c56",
                plot_bgcolor = "#1f2c56",
                height=40,
#                 shape
                
                )
            }

# Donut chart
@app.callback(
	Output(
		component_id = "pie_chart",
		component_property = "figure"
	),[
	Input(component_id = "w_countries",component_property = "value"),
	Input('column_select', 'value'),
	Input('date_slider', 'value')
    ]
)
def update_pie_chart(countries,column,date_index):
	value = []
	date = covid_data['date'].unique()[date_index]
	covid_data_filtered = covid_data[covid_data['date'] <= date]
	date_lst = covid_data_filtered['date']
	# List of colors
	colors = ["orange", "#dd1e35", "green", "#e55467"] 
	covid_data_2 = covid_data_filtered.groupby(["date", "Country/Region"])[["confirmed", "deaths", "recovered", "active"]].sum().reset_index()
	for country in countries:
		if column == "confirmed":
			value.append(covid_data_2[covid_data_2["Country/Region"] == country]["confirmed"].sum())
		elif column == "deaths":
			value.append(covid_data_2[covid_data_2["Country/Region"] == country]["deaths"].sum())
		elif column == "recovered":
			value.append(covid_data_2[covid_data_2["Country/Region"] == country]["recovered"].sum())
		else:
			value.append(covid_data_2[covid_data_2["Country/Region"] == country]["active"].sum())

	# Build the figure
	fig = {
		"data": [
			go.Pie(
			labels = countries,
			values = value,
			marker = {
				"colors": colors
			},
			hoverinfo = "label+value+percent",
			textinfo = "label+value",
			hole = 0.7,
			rotation = 45,
			insidetextorientation = "radial"
			)
        ],
		"layout": go.Layout(
			title = {
				"text": "Total "+ column + " cases", 
#\n [" + str(date_lst.iloc[0]).split(" ")[0] + " : " + str(date_lst.iloc[-1]).split(" ")[0] + "]"
				"y": 0.93,
				"x": 0.5,
				"xanchor": "center",
				"yanchor": "top"
			},
			titlefont = {
				"color": "white",
				"size": 17
			},
			font = {
				"family": "sans-serif",
				"color": "white",
				"size": 12
			},
			hovermode = "closest",
			paper_bgcolor = "#1f2c56",
			plot_bgcolor = "#1f2c56",
			legend = {
				"orientation": "h",
				"bgcolor": "#1f2c56",
				"xanchor": "center",
				"x": 0.5,
				"y": -0.7
			}
		)
	}
	# Return the figure
	return fig




# Line and bars chart
@app.callback(
	Output(
		component_id = "line_chart",
		component_property = "figure"
	),[
	Input(
		component_id = "w_countries",
		component_property = "value"
	),
    Input('column_select', 'value'),
    Input('date_slider', 'value')
    ]
)

def update_line_chart(countries, column,date_index):
    date = covid_data['date'].unique()[date_index]
    covid_data_filtered = covid_data[covid_data['date'] <= date]
    traces = []
    countries = covid_data_filtered[(covid_data_filtered['Country/Region'].isin(countries)) &
                   (covid_data_filtered['date'] == covid_data_filtered['date'].max())].groupby('Country/Region')['confirmed'].sum().sort_values(ascending=False).index.to_list()
    for country in countries:
        
        y_data = covid_data_filtered[covid_data_filtered['Country/Region'] == country].groupby('date')[column].sum()
        traces.append(go.Scatter(
                    x=covid_data_filtered[covid_data_filtered['Country/Region'] == country].groupby('date')['date'].first(), 
                    y=y_data,
#                     hovertemplate=hover,   
                    name=country,
                    mode='lines'))
    return {
            'data': traces,
            'layout': go.Layout(
                    title="{} by Region".format(column),
                    xaxis_title="Date",
                    yaxis_title="Number of Cases",
                    font=dict(color=dash_colors['text']),
                    paper_bgcolor="#1f2c56",
                    plot_bgcolor="#1f2c56",
                    xaxis=dict(gridcolor=dash_colors['grid']),
                    yaxis=dict(gridcolor=dash_colors['grid']),
                    hovermode='closest'
                )
            }


# Map
@app.callback(
	Output(
		component_id = "map_chart",
		component_property = "figure"
	),
	Input(
		component_id = "w_countries",
		component_property = "value"
	)
)
def update_map(w_countries):
	lat_lst = []
	long_lst = []
	confirmed_lst = []
	# Filter the data
	covid_data_4 = covid_data.groupby(["Lat", "Long", "Country/Region"])[["confirmed", "deaths", "recovered", "active"]].max().reset_index()
	covid_data_5 = covid_data_4[covid_data_4["Country/Region"].isin(w_countries)]
	fig = {
		"data": [
			go.Scattermapbox(
				lon = covid_data_5['Long'],
				lat = covid_data_5['Lat'],
				mode = "markers",
				marker = go.scattermapbox.Marker(
					size = covid_data_5["confirmed"] / 200,
					color = "orange",
					colorscale = "HSV",
					showscale = False,
					sizemode = "area",
					opacity = 0.3
				),
				hoverinfo = "text",
				hovertemplate = "<b>Country:</b> " + covid_data_5["Country/Region"].astype(str) + "<br>" +
												"<b>Confirmed cases:</b> " + [f'{x:,.0f}' for x in covid_data_5["confirmed"]] + "<br>" + 
												"<b>Deaths:</b> " + [f'{x:,.0f}' for x in covid_data_5["confirmed"]] + "<br>" + 
												"<b>Recovered:</b> " + [f'{x:,.0f}' for x in covid_data_5["recovered"]] + "<br>" + 
												"<b>Active:</b> " + [f'{x:,.0f}' for x in covid_data_5["active"]] + "<extra></extra>"
			)
		],
		"layout": go.Layout(
			hovermode = "x",
			paper_bgcolor = "#1f2c56",
			plot_bgcolor = "#1f2c56",
			margin = {
				"r": 0,
				"l": 0,
				"t": 0,
				"b": 0
			},
			mapbox = dict(
				accesstoken = "pk.eyJ1IjoicXM2MjcyNTI3IiwiYSI6ImNraGRuYTF1azAxZmIycWs0cDB1NmY1ZjYifQ.I1VJ3KjeM-S613FLv3mtkw",
# 				center = go.layout.mapbox.Center(
# 					lat = zoom_lat,
# 					lon = zoom_long
# 				),
				style = "dark",
# 				zoom = zoom
			),
			autosize = True
		)
	}
	# Return the figure
	return fig

# Run the app

app.run_server(debug = True)


<ipython-input-11-3763430bae65>:74: UserWarning:

DataFrame columns are not unique, some columns will be omitted.



Dash app running on http://127.0.0.1:8050/
